In [7]:
import numpy as np
import pandas as pd
df = pd.read_csv('dados/nova_plataforma.csv')

In [8]:
import numpy as np
import pandas as pd

# Carregar os dados originais
df = pd.read_csv('imputed_plataformas_otc.csv')

# Colunas conforme categorizado
colunas_float = ['height_of_jacket_or_sub-structure (m)']

colunas_int = ['water_depth (m)', 'installation_date', 'weight (t)', 'number_of_legs', 'number_of_piles', 
               'distance_to_coast (km)', 'energy_consumption-complete (GJ)', 'energy_consumption-partial (GJ)', 
               'emissions-complete (t)', 'emissions-partial (t)', 'type_of_production (1 oil and gas; 2 oil; 3 gas)', 'recommended (1 partial; 2 complete)']

# Combinar colunas numéricas (floats e inteiros) para calcular a matriz de covariância
colunas_numericas = colunas_float + colunas_int

# Calcular a média e a matriz de covariância das colunas numéricas
means_numericas = df[colunas_numericas].mean()
cov_numericas = df[colunas_numericas].cov()

# Gerar dados sintéticos para as colunas numéricas usando a distribuição normal multivariada
dados_sinteticos_numericos = np.random.multivariate_normal(mean=means_numericas, cov=cov_numericas, size=10000)

# Concatenar todos os dados gerados
dados_sinteticos = dados_sinteticos_numericos

# Criar o DataFrame final com os dados sintéticos
colunas_sinteticas = colunas_numericas
df_sintetico = pd.DataFrame(dados_sinteticos, columns=colunas_sinteticas)

# Ajustar colunas que não podem ter valores negativos
df_sintetico[colunas_int] = df_sintetico[colunas_int].clip(lower=0)

# Definir valores máximos/lógicos para colunas específicas
df_sintetico['number_of_legs'] = df_sintetico['number_of_legs'].clip(lower=4, upper=12)
df_sintetico['installation_date'] = df_sintetico['installation_date'].clip(lower=13, upper=50)

# Salvando os dados sintéticos gerados
df_sintetico.to_csv('registros_distribuicao.csv', index=False)


In [9]:
import pandas as pd
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

# Carregar os dados originais e sintéticos
df_original = pd.read_csv('imputed_plataformas_otc.csv')  # Seu dataset original
df_sintetico = pd.read_csv('registros_distribuicao.csv')  # Dados sintéticos gerados

metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "recommended (1 partial; 2 complete)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade entre os dados reais e sintéticos
report = QualityReport()
report.generate(real_data=df_original, synthetic_data=df_sintetico, metadata=metadata_dict)

# Exibir detalhes específicos do relatório
print(report.get_details(property_name='Column Shapes'))
print(report.get_details(property_name='Column Pair Trends'))


Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 13/13 [00:00<00:00, 456.48it/s]|
Column Shapes Score: 71.81%



(2/2) Evaluating Column Pair Trends: |██████████| 78/78 [00:00<00:00, 121.75it/s]|
Column Pair Trends Score: 99.43%

Overall Score (Average): 85.62%

                                              Column        Metric   Score
0                                    water_depth (m)  KSComplement  0.6826
1                                         weight (t)  KSComplement  0.6956
2                                  installation_date  KSComplement  0.7959
3   type_of_production (1 oil and gas; 2 oil; 3 gas)  KSComplement  0.6487
4                                     number_of_legs  KSComplement  0.6173
5                                    number_of_piles  KSComplement  0.7067
6              height_of_jacket_or_sub-structure (m)  KSComplement  0.6775
7                             distance_to_coast (km)  KSComplement  0.7950
8                   energy_consumption-complete (GJ)  KSComplement  0.8183
9                    energy_consumption-partial (GJ)  KSComplement  0.7098
10                       